In [130]:
#What is the probability that I will get booked if I charge $35/night with 4 repeat guests and 10 reviews?

In [1]:
import pandas as pd
import statsmodels.api as sm
import math
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.svm import SVC
from os.path import join
from sklearn import metrics

In [2]:
df = pd.read_csv('dog-vacay.csv')

In [3]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
df['fee35'] = df['fee'].map(lambda x: 1 if x >= 35 else 0)

In [5]:
df.head()

,fee,names,ratings,repeat,response_time,review,services,title,boarding,sitting,daycare,checkups,walking,bathing,training,grooming,pickup/dropoff,fee35
0,25,patricia's pet heaven,4.5,9,1,11,1,"Phoenix, Arizona",1,0,0,0,0,0,0,0,0,0
1,35,**DOGGIE BED & BARKFEST**,5.0,1,2,45,3,"Phoenix, Arizona",1,0,1,0,0,1,0,0,0,1
2,20,Doggie Heaven,5.0,1,3,28,4,"Phoenix, Arizona",1,1,1,1,0,0,0,0,0,0
3,25,Stay at home mother and dog lover!,5.0,5,2,6,4,"Phoenix, Arizona",1,0,1,1,1,0,0,0,0,0
4,20,Kelrey's Doggy Dream Stay,4.5,3,1,3,4,"Phoenix, Arizona",1,0,1,0,0,1,0,0,1,0


In [6]:
cols = ['fee', 'ratings', 'repeat', 'response_time', 'review', 'services']

In [7]:
x = df[cols]
y = df['fee35']

In [8]:
#Define the logistic regression model.
model = sm.Logit(y,x)

In [9]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.401746
         Iterations 7


In [10]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  fee35   No. Observations:                 7921
Model:                          Logit   Df Residuals:                     7915
Method:                           MLE   Df Model:                            5
Date:                Sat, 02 Apr 2016   Pseudo R-squ.:                  0.3959
Time:                        11:21:57   Log-Likelihood:                -3182.2
converged:                       True   LL-Null:                       -5267.3
                                        LLR p-value:                     0.000
=================================================================================
                    coef    std err          z      P>|z|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
fee               0.1885      0.004     42.967      0.000         0.180     0.197
ratings          -0.1335      0.015     -8.772      0.000        -0.163    -0.104
repeat           -0.0047      0.018     -0.255      0.799        -0.041     0.031
response_time    -0.6879      0.047    -14.624      0.000        -0.780    -0.596
review           -0.0006      0.002     -0.301      0.764        -0.004     0.003
services         -1.2998      0.035    -37.336      0.000        -1.368    -1.232
=================================================================================
"""

In [11]:
def logistic_function(fee, ratings, repeat, response_time, review, services ):
    prob = 1/(1 + math.exp(coeff[0]*repeat+coeff[1]*review))
    return prob

In [12]:
coeff = result.params
print coeff

fee              0.188506
ratings         -0.133494
repeat          -0.004704
response_time   -0.687879
review          -0.000565
services        -1.299838
dtype: float64


In [13]:
logistic_function(30, 5, 5, 3, 5, 3)

0.43166600856542703

In [14]:
#SVC TEST

In [15]:
dfTrain, dfTest = train_test_split(df, test_size=0.1, random_state=0)

In [16]:
dfTrain = pd.DataFrame(dfTrain, columns=df.columns)
dfTest = pd.DataFrame(dfTest, columns=df.columns)

In [17]:
x1 = dfTrain[cols]
y1 = dfTrain['fee35']

In [18]:
#Train
clf = SVC(gamma=.001)#can change #

In [19]:
clf.fit(x1,y1)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

In [20]:
#Test
x2 = dfTest[cols]
y2 = dfTest['fee35']

In [21]:
ypred = clf.predict(x2)

In [22]:
#Converts the array to one with data type np.float64 (i.e. an array where the array elements 
#are the actual floating point values, not pointers to objects)
a = ypred.astype(np.float64)

In [23]:
fpr, tpr, thresholds = metrics.roc_curve(y2, a, pos_label=1)
AUC = metrics.auc(fpr, tpr)

In [24]:
AUC

0.99096385542168675